# Shoe Average Order Values

On an e-commerce engine, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# path = "https://docs.google.com/spreadsheets/d/16i38oonuX1y1g7C_UAmiK9GkY7cS-64DfiDMNiR41LM/edit#gid=0"

In [3]:
!head -n 5 data/Shoes_AOV_data.csv

In [5]:
shoes = pd.read_csv('data/Shoes_AOV_data.csv', index_col="order_id")
shoes.head(5)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56
2,92,925,90,1,cash,2017-03-03 17:38:52
3,44,861,144,1,cash,2017-03-14 4:23:56
4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [6]:
shoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shop_id         5000 non-null   int64 
 1   user_id         5000 non-null   int64 
 2   order_amount    5000 non-null   int64 
 3   total_items     5000 non-null   int64 
 4   payment_method  5000 non-null   object
 5   created_at      5000 non-null   object
dtypes: int64(4), object(2)
memory usage: 273.4+ KB


In [7]:
shoes.shop_id = shoes.shop_id.astype("category")
shoes.user_id = shoes.user_id.astype("category")
shoes.created_at = pd.to_datetime(shoes.created_at)

In [8]:
shoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   shop_id         5000 non-null   category      
 1   user_id         5000 non-null   category      
 2   order_amount    5000 non-null   int64         
 3   total_items     5000 non-null   int64         
 4   payment_method  5000 non-null   object        
 5   created_at      5000 non-null   datetime64[ns]
dtypes: category(2), datetime64[ns](1), int64(2), object(1)
memory usage: 225.3+ KB


In [9]:
shoes.order_amount.mean()

3145.128

**a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.**

Let's start by looking at a little more of the data to see if anything jumps out.

In [10]:
shoes.head(20)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56
2,92,925,90,1,cash,2017-03-03 17:38:52
3,44,861,144,1,cash,2017-03-14 04:23:56
4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,18,883,156,1,credit_card,2017-03-01 04:35:11
6,58,882,138,1,credit_card,2017-03-14 15:25:01
7,87,915,149,1,cash,2017-03-01 21:37:57
8,22,761,292,2,cash,2017-03-08 02:05:38
9,64,914,266,2,debit,2017-03-17 20:56:50


Whoah, order_id = 16 had order_amount = 704000 and total_items = 2000. That huge outlier is going to drive the mean way up. Let's look at some more statistics to see what we can see. 

In [11]:
shoes.describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


Max order amount is 704,000. That will have huge impact on the mean. The median is 284 and the median total items is 2, meaning roughly ~140 per pair of shoes, closer to what we would expect.

**b. What metric would you report for this dataset?**

**c. What is its value?**

As a start, I would report the median order value, whose value is $284

Let's look more closely at the huge orders

In [12]:
shoes_outliers = shoes[shoes.order_amount > 1000]
shoes_outliers.head()

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
16,42,607,704000,2000,credit_card,2017-03-07 04:00:00
61,42,607,704000,2000,credit_card,2017-03-04 04:00:00
161,78,990,25725,1,credit_card,2017-03-12 05:56:57
491,78,936,51450,2,debit,2017-03-26 17:08:19
494,78,983,51450,2,cash,2017-03-16 21:39:35


In [26]:
shoes_outliers.tail()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00
4715,4716,78,818,77175,3,debit,2017-03-05 5:10:44
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00:00
4882,4883,42,607,704000,2000,credit_card,2017-03-25 4:00:00
4918,4919,78,823,25725,1,cash,2017-03-15 13:26:46


In [13]:
shoes_outliers.shape

(71, 6)

In [15]:
shoes_outliers.head(20)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
16,42,607,704000,2000,credit_card,2017-03-07 04:00:00
61,42,607,704000,2000,credit_card,2017-03-04 04:00:00
161,78,990,25725,1,credit_card,2017-03-12 05:56:57
491,78,936,51450,2,debit,2017-03-26 17:08:19
494,78,983,51450,2,cash,2017-03-16 21:39:35
512,78,967,51450,2,cash,2017-03-09 07:23:14
521,42,607,704000,2000,credit_card,2017-03-02 04:00:00
618,78,760,51450,2,cash,2017-03-18 11:18:42
692,78,878,154350,6,debit,2017-03-27 22:51:43


In [14]:
shoes_outliers.shop_id.unique()

[42, 78, 43, 54]
Categories (4, int64): [42, 78, 43, 54]

So we have four stores with orders of over 1000 dollars. From our peek at the data, we can see that shop 42 has a bunch of 704,000 dollar orders of 2000 total items. 2000 pair of 352 dollar shoes. Military boots, athletic gear? Shop 78 has shoes costing 25,725 dollars. Rare, collectible sneakers? Flats and heels encrusted with diamonds? Let's look at the other two stores with orders over 1000 dollars

In [16]:
shoes_outliers[shoes_outliers.shop_id == 43]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
3539,43,830,1086,6,debit,2017-03-17 19:56:29


In [17]:
shoes_outliers[shoes_outliers.shop_id == 54]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
4142,54,733,1064,8,debit,2017-03-07 17:05:18


OK, so the last two, shop 43 and shop 54, aren't quite as unusual. Orders of 6 and 8 pairs that barely crept over 1000 dollars. So let's look at the data after removing shops 42 and 78.

In [33]:
shoes_no_42_or_78 = shoes[shoes.shop_id != 78]
shoes_no_42_or_78 = shoes_no_42_or_78[shoes.shop_id != 42]

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [34]:
shoes_no_42_or_78.shape

(4903, 6)

In [35]:
shoes_no_42_or_78.head(20)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56
2,92,925,90,1,cash,2017-03-03 17:38:52
3,44,861,144,1,cash,2017-03-14 04:23:56
4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,18,883,156,1,credit_card,2017-03-01 04:35:11
6,58,882,138,1,credit_card,2017-03-14 15:25:01
7,87,915,149,1,cash,2017-03-01 21:37:57
8,22,761,292,2,cash,2017-03-08 02:05:38
9,64,914,266,2,debit,2017-03-17 20:56:50


In [37]:
shoes_no_42_or_78.order_amount.mean(), shoes_no_42_or_78.total_items.mean()

(300.1558229655313, 1.9957169080155006)

In [38]:
shoes_no_42_or_78.describe()

,order_amount,total_items
count,4903.000000,4903.000000
mean,300.155823,1.995717
std,155.941112,0.982602
min,90.000000,1.000000
25%,163.000000,1.000000
50%,284.000000,2.000000
75%,386.500000,3.000000
max,1086.000000,8.000000


Simply dropping shops 42 and 78 gets us to a data set with 4903 entries out of 5000, with an average order value of 300 for 2 items. This is pretty close to our median of the original data set at 284 for 2 items. The data is still just a little skewed, as there are more orders for one item and the orders for 6 or 8 items drive the mean up just a tad. Still fairly close. However there is a significant reason why this is not the best approach: we are losing information about the revenue of the two shoe stores that have the largest revenues.

In [45]:
sum(shoes[shoes.shop_id == 42].order_amount) / sum(shoes.order_amount)

0.7624602877847897

In [46]:
sum(shoes[shoes.shop_id == 78].order_amount) / sum(shoes.order_amount)

0.1439559852571978

Shop 42 accounted for over 75% of the total revenue and shop 78 another 14%. Together, these two shops accounted for over 90% of the total of our orders. 